---

**Исследование надежности заемщиков**

*Описание проекта*

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Загрузим таблицу.

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

Выведем информацию по таблице.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*Описание данных*

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
*dob_years — возраст клиента в годах
*education — уровень образования клиента
*education_id — идентификатор уровня образования
*family_status — семейное положение
*family_status_id — идентификатор семейного положения
*gender — пол клиента
*income_type — тип занятости
*debt — имел ли задолженность по возврату кредитов
*total_income — ежемесячный доход
*purpose — цель получения кредита

В столбцах `days_employed` и `total_income` есть пропущенные значения, причем их количество - 2174 (10% от общего количества), равно в обоих столбцах. Очевидно это произошло при подгрузке части данных.

Выведем первые 10 строк таблицы.

In [3]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем статестические данные по столбцам с числами.


In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Просматривая статистическую таблицу видим, что в столбцах `children` и `days_employed` присутствуют отрицательные значения. Очевидно это ошибка связана с человеческим фактором. Минус можно "отбросить".
Так же в `days_emploeyed` невероятно высокое максимальное значение `401755.4` дня трудавого стажа. Это `1100` лет. Тут явно опечатка связанная с человеческим фактором.
В столбце `dob_years` есть нулевые значения. Возраст клиента не может быть равен 0. Вероятно ноль ставился вместо пустых значений. Их нужно заменить на медианное значение. В столбце `total_income` так же заменим пропуски медианным значением.

>Медиана поможет найти именно тот показатель, который ближе всего к истинно среднему. На неё не влияют выбивающиеся из общей массы числа, поэтому она считается одним из самых надёжных и устойчивых показателей.

Посмотрим на строки с отрицательным трудовым стажем. Выведим их на экран.

In [5]:
display(data[data['days_employed'] <= 0].sort_values(by='days_employed'))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
2127,1,-34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования
9683,0,-33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
6157,2,-30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
8336,0,-24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования


Очевидно что данные клиентов о трудовом стаже были введены со знаком минус. Принимаем решение убрать эти отрицательные значения.

In [6]:
#избавимся от отрицательных значений
data['days_employed'] = data['days_employed'].abs()

#высчитаем медианное значение столбца
days_employed_median = data['days_employed'].median()

#заменим пустые значения на медиану
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

#переведем трудовой стаж в днях на стаж в годах, т.к. это более привычно
data['days_employed'] = data['days_employed'] / 365.25

#переименуем столбец на 'years_employed'
data = data.rename(columns = {'days_employed':'years_employed'})

Исследуем очень большие значения трудавого стажа

In [7]:
display(data[data['years_employed'] > 50].sort_values(by='years_employed'))

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,50.346201,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
20444,0,900.010186,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,900.027170,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,900.126876,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,900.193639,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
7794,0,1099.695688,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,1099.724755,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,1099.726471,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,1099.837951,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования


Не знаю чем вызванна это аномалия. У 3446 клиентов из 21525 стаж более 900 лет. Это у 16% от общего числа. Чтобы уменьшить влияние этой анамалии присвоим всем таким клиентам присвоим медийное значение трудавого стажа.

In [8]:
#высчитаем медиану
days_employed_median = data['years_employed'].median()

#всем клиентам со стажем более 60 лет присваиваем медианное значение стажа
data.loc[data['years_employed'] > 60, 'years_employed'] = days_employed_median

#выведим последние 10 строк таблицы
display(data['years_employed'].sort_values().tail(10))

15675    42.190392
7731     42.759928
4321     43.184288
1539     43.218833
3974     43.355854
16825    44.133300
17838    44.530320
7329     45.430453
4299     48.228784
16335    50.346201
Name: years_employed, dtype: float64

Исследуем столбец `children`. При просмотре статистики о датафрейме, было видно, что там есть и отрицательные значения и max=20.

In [9]:
#посмотрим сколько клиентов имеют более 5 детей
display(data[data['children']>5].sort_values(by='children'))

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,2.409914,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
15976,20,6.007449,39,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,покупка недвижимости
15812,20,9.412852,42,среднее,1,женат / замужем,0,F,сотрудник,0,295265.213511,жилье
15762,20,12.988749,41,среднее,1,женат / замужем,0,F,сотрудник,0,143529.459627,дополнительное образование
15674,20,14.838768,33,среднее,1,гражданский брак,1,F,сотрудник,0,155567.570284,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
8430,20,6.007449,60,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,покупка жилья
7962,20,0.886241,40,высшее,0,женат / замужем,0,M,компаньон,0,190051.666340,строительство собственной недвижимости
7789,20,6.007449,55,среднее,1,женат / замужем,0,F,пенсионер,0,138626.756600,строительство собственной недвижимости
10194,20,17.953038,42,высшее,0,женат / замужем,0,M,сотрудник,0,124621.997936,образование


76 клиентов из 21525 имеют 20 детей. Такая ошибка явно из-за человеческого фактора. Так как в исследовании надо найти зависимость между количеством детей и возвратом кредита в срок, эти данные для нас важны. Но из-за того, что мы не знаем как их интерпретировать, нам придется удалить эти строки и проиндексировать таблицу заново.

In [10]:
#удаляем значения и переиндексируем таблицу
data = data.loc[data['children'] != 20].reset_index(drop=True)

#уберем отрицательные значения в этом солбце
data['children'] = data['children'].abs()

#проверим что у нас получилось выведя статестическую таблицу
data.describe()

,children,years_employed,dob_years,education_id,family_status_id,debt,total_income
count,21449.000000,21449.000000,21449.000000,21449.000000,21449.000000,21449.000000,1.928400e+04
mean,0.474334,6.330221,43.298615,0.817148,0.973099,0.080796,1.674178e+05
std,0.751417,5.425602,12.576314,0.548507,1.420521,0.272528,1.030418e+05
min,0.000000,0.066096,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,2.815060,33.000000,1.000000,0.000000,0.000000,1.030161e+05
50%,0.000000,6.007449,42.000000,1.000000,0.000000,0.000000,1.450208e+05
75%,1.000000,6.894371,53.000000,1.000000,1.000000,0.000000,2.033700e+05
max,5.000000,50.346201,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбце `dob_years` есть нулевые значения. Клиенты наверника просто не указали свой возраст. Заменим их на медианные.


In [11]:
#высчитываем медианное значение и заменяем им нулевые
days_dob_median = data['dob_years'].median()
data.loc[data['dob_years'] == 0, 'dob_years'] = days_dob_median

В столбце `total_income`заменим пустые значения на медианные.

In [12]:
#находим медиану
data_income_median = data['total_income'].median()
#заменяем 
data['total_income'] = data['total_income'].fillna(data_income_median)
#проверяем
data.describe()

,children,years_employed,dob_years,education_id,family_status_id,debt,total_income
count,21449.000000,21449.000000,21449.000000,21449.000000,21449.000000,21449.000000,2.144900e+04
mean,0.474334,6.330221,43.494429,0.817148,0.973099,0.080796,1.651571e+05
std,0.751417,5.425602,12.222610,0.548507,1.420521,0.272528,9.793553e+04
min,0.000000,0.066096,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,2.815060,34.000000,1.000000,0.000000,0.000000,1.077720e+05
50%,0.000000,6.007449,42.000000,1.000000,0.000000,0.000000,1.450208e+05
75%,1.000000,6.894371,53.000000,1.000000,1.000000,0.000000,1.955269e+05
max,5.000000,50.346201,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Заменим вещественный тип данных в столбцах `total_incom` на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype('int')

#выведем информацию на экран
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21449 entries, 0 to 21448
Data columns (total 12 columns):
children            21449 non-null int64
years_employed      21449 non-null float64
dob_years           21449 non-null float64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null int64
purpose             21449 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**С артефактами и пропусками разобрались. Приступаем к дубликатам.**

In [14]:
display(data.head(10))

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.101090,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11.019312,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15.396092,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11.292942,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,6.007449,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,2.535759,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,7.882826,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,0.418288,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18.972937,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,5.992489,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


В столбце `education` есть данные записанные с большой буквы. Переведем все символы в этом столбце к нижнему регистру.

In [15]:
#переводим столбец к нижнему регистру и выводим уникальные значения
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

В этом столбце все исправили. Проверим другие.

In [16]:
#выводим уникальные значения в столбце `family_status`.
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Здесь так же потребуется привести одно из значений к нижнему регистру.

In [17]:
data['family_status'] = data['family_status'].str.lower()

In [18]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Теперь все хорошо. Переходим к следующему.

In [19]:
#выводим уникальные значения в столбце `income_type`.
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Тут все в порядке. Переходим к следующему.

In [20]:
#выводим уникальные значения в столбце `purpose`.
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Все значения в нижнем регистре, но в этом столбце очень много неявных дубликатов. 
Такое большое количество дубликатов говорит о том, что графу "Цель получения кредита" заполняют без использования шаблона. 
В анкете нет готовых ответов из которых надо выбрать нужный и каждый клиент банка заполняет эту графу по своему усмотрению.

**Категоризация данных**

Напишем функцию, которая на основании данных из столбца `purpose` формирует новый столбец `purpose_category`, в который входит следующие категории:
* `операции с автомобилем`
* `операции с недвижимостью`
* `проведение свадьбы`
* `получение образования`

При написании функции введем в нее операторы `try:` `except:` для возможной обработки ошибки если такого значения не найдется в таблице.

In [21]:
def selection_purpose (purpose):
    
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'свадьб'in purpose:
        return 'проведение свадьбы'
    elif 'образовани' in purpose:
        return 'получение образования'
    else:
        try:
            return 'операции с недвижимостью'
        except:
            print('категория не определена')

Пременим метод `.apply()` и вставим столбец `purpose_category` в датафрейм.

In [22]:
data['purpose_category'] = data['purpose'].apply(selection_purpose)

Создадим два датафрейма со столбцами:
* `education_id` и `education` - в первом;
* `family_status_id` и `family_status` - во втором.

Удалим дубликаты и переиндексирум по первому столбцу.

In [23]:
data_education = data[['education_id', 'education']].drop_duplicates().set_index('education_id')

data_family = data[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')
# проверим что получилось
display(data_education)
display()
display(data_family)

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,не женат / не замужем


Удаляем из исходного датафрейма столбцы `education` и `family_status`, оставив только их индентификаторы `education_id` и `family_status_id`. Удалим так же и столбец `purpose` т.к. мы ренее получили на его основе столбец `purpose_category`.

In [24]:
data = data.drop(['education', 'family_status', 'purpose'], axis=1)

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
*     0 - 30000 -   'E';
* 30001 - 50000 -   'D';
* 50001 - 200000 -  'C';
* 200001 - 1000000 - 'B';
* 1000001 и выше -  'A'.

Напишем функцию

In [25]:
def income_category (income):
    if  0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50000 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'
    else:
        return 'Что-то пошло не так'

Созданим новый столбец `total_income_category` применив метод .apply()

In [26]:
data['total_income_category'] = data['total_income'].apply(income_category)
#посмотрим что получилось
display(data.head(10))

,children,years_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category,total_income_category
0,1,23.101090,42.0,0,0,F,сотрудник,0,253875,операции с недвижимостью,B
1,1,11.019312,36.0,1,0,F,сотрудник,0,112080,операции с автомобилем,C
2,0,15.396092,33.0,1,0,M,сотрудник,0,145885,операции с недвижимостью,C
3,3,11.292942,32.0,1,0,M,сотрудник,0,267628,получение образования,B
4,0,6.007449,53.0,1,1,F,пенсионер,0,158616,проведение свадьбы,C
5,0,2.535759,27.0,0,1,M,компаньон,0,255763,операции с недвижимостью,B
6,0,7.882826,43.0,0,0,F,компаньон,0,240525,операции с недвижимостью,B
7,0,0.418288,50.0,1,0,M,сотрудник,0,135823,получение образования,C
8,2,18.972937,35.0,0,1,F,сотрудник,0,95856,проведение свадьбы,C
9,0,5.992489,41.0,1,0,M,сотрудник,0,144425,операции с недвижимостью,C


**Исследуем если зависимость между количеством детей и возвратом кредита в срок.**

Создадим сводную таблицу.

In [27]:
pivot_children = data.pivot_table(index='children', columns='debt', values='gender', aggfunc='count')
#выведем на экран
pivot_children

debt,0,1
children,,
0,13086.0,1063.0
1,4420.0,445.0
2,1861.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Посчитаем процент невозврата и индекс доверия. 
Индекс доверия это обратное соотношение процента невозврата кредита к его медианному значению.

In [28]:
#считаем процент невозврата и присоединяем столбец
pivot_children['percent'] = pivot_children[1] / pivot_children[0] * 100
#посчитаем медианное значение 
pivot_children_median = pivot_children['percent'].median()
#считаем индекс ловерия и присоединяем к таблице
pivot_children['trust'] = pivot_children_median / pivot_children['percent']
#выведем получившуюся таблицу
display(pivot_children)

debt,0,1,percent,trust
children,,,,
0,13086.0,1063.0,8.123185,1.239400
1,4420.0,445.0,10.067873,1.000000
2,1861.0,194.0,10.424503,0.965789
3,303.0,27.0,8.910891,1.129839
4,37.0,4.0,10.810811,0.931278
5,9.0,NaN,NaN,NaN


Анализируя полученные данные видим, что клиенты без детей самые надежные +24%. Это и понятно, не имея детей легче расплатится за кредит.
Так же хорошими клиентами являются обладатели 3 детей +13%. С чем это связано - непонятно. И почему клиенты с 2 и 4 детьми наоборот, являются менее надежными клиентами -4% и -7% соответственно. Этот вопрос может быть дополнительно изучен (за пределами данного проекта).

**Исследуем зависимость между семейным положением и возврата кредита в срок.**

Создадим сводную таблицу по семейному положению.

In [29]:
pivot_family_status = data.pivot_table(index='family_status_id', columns='debt', values='gender', aggfunc='count')
pivot_family_status

debt,0,1
family_status_id,,
0,11403,928
1,3780,385
2,893,63
3,1109,84
4,2531,273


Посчитаем процент невозврата и индекс доверия.

In [30]:
pivot_family_status['percent'] = pivot_family_status[1] / pivot_family_status[0] * 100
#посчитаем медианное значение и заведем столбец
pivot_family_status_median = pivot_family_status['percent'].median()
pivot_family_status['trust'] = pivot_family_status_median / pivot_family_status['percent']
#выведем получившуюся таблицу
pivot_family_status

debt,0,1,percent,trust
family_status_id,,,,
0,11403,928,8.138209,1.000000
1,3780,385,10.185185,0.799024
2,893,63,7.054871,1.153559
3,1109,84,7.574391,1.074437
4,2531,273,10.786250,0.754498


Расшифруем `family_status_id` произведя слияние методом `merge()` с таблицей `data_family`.

In [31]:
pivot_family_status = pivot_family_status.merge(data_family, on='family_status_id', how='left')

In [32]:
#выведем на экран
pivot_family_status

,0,1,percent,trust,family_status
family_status_id,,,,,
0,11403,928,8.138209,1.000000,женат / замужем
1,3780,385,10.185185,0.799024,гражданский брак
2,893,63,7.054871,1.153559,вдовец / вдова
3,1109,84,7.574391,1.074437,в разводе
4,2531,273,10.786250,0.754498,не женат / не замужем


Проанализируем полученные значения. 
Самый лучший клиент для банка это `вдовец/вдова`+15% и кто `в разводе`+7%. Самый худший `не женат\не замужем` - 25% и те у кого `гражданский брак`-20%.
Интересная законамерность. Люди которые были в браке, но сейчас одни, являются самыми надежными клиентами. Возможно они понимают, что нужно надеятся только на себя и подходят к возврату кредита более ответственно.
Те же, кто в гражданском браке или же не женат/не замужем, позволяют себе просрочки по кредиту.

**Исследуем зависимость между уровнем дохода и возвратом кредита в срок.**

Создадим сводную таблицу.

In [33]:
pivot_income = data.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')
#выведем на экран
pivot_income

debt,0,1
total_income_category,,
A,23,2
B,4667,354
C,14677,1354
D,329,21
E,20,2


Посчитаем процент невозврата и индекс доверия.

In [34]:
pivot_income['percent'] = pivot_income[1] / pivot_income[0] * 100
#посчитаем медианное значение и заведем столбец отношение процента невозврата кредита к медианному значению
pivot_income_median = pivot_income['percent'].median()
pivot_income['trust'] = pivot_income_median / pivot_income['percent']
#присоединим солбец с расшифровкой категорий
pivot_income['category'] = ['более 1000001', 'от 200001 до 1000000', 'от 50001 до 200000', 'от 30001 до 50000', 'до 30000']
#выведем получившуюся таблицу
pivot_income

debt,0,1,percent,trust,category
total_income_category,,,,,
A,23,2,8.695652,1.000000,более 1000001
B,4667,354,7.585172,1.146401,от 200001 до 1000000
C,14677,1354,9.225319,0.942586,от 50001 до 200000
D,329,21,6.382979,1.362319,от 30001 до 50000
E,20,2,10.000000,0.869565,до 30000


Проанализируем полученные значения. Самыми надежными клиентами банка составляют клиенты из категории `D` +36% и категории `B`+15%, а самые ненадежные клиенты из категории `E` -13% и из категории `C` -6%.
Высокий процент невозврата кредита в срок у клиентов категории `E`, с малым доходом, объясняется спонтанными покупками.
Клиенты с доходом из категории `D` имея средний доход, очень ответственно подходят к возврату кредита. Предположу, что это люди старшего поколения (эту теорию можно проверить за рамками данного проекта).
У следущей категории `C`, наблюдается спад индекса доверия. Предположу это люби в высокой зарплатой, но низкой кредитной культурой. (Было бы интересно проверить как это коррелируется с возрастом клиентов).
У клиентов `B` и `A` высокая зарплата и у них нет сложностей с возвратом кредита с срок.

**Исследуем как разные цели кредита влияют на его возврат в срок.**

Создадим сводную таблицу.

In [35]:
pivot_purpose = data.pivot_table(index='purpose_category', columns='debt', values='gender', aggfunc='count')
pivot_purpose

debt,0,1
purpose_category,,
операции с автомобилем,3898,401
операции с недвижимостью,10024,780
получение образования,3638,369
проведение свадьбы,2156,183


Расчитаем процент невозврата кредита и индекс доверия.

In [36]:
pivot_purpose['percent'] = pivot_purpose[1] / pivot_purpose[0] * 100
#посчитаем медианное значение и заведем столбец отношение процента невозврата кредита к медианному значению
pivot_purpose_median = pivot_purpose['percent'].median()
pivot_purpose['trust'] = pivot_purpose_median / pivot_purpose['percent']
#выведем получившуюся таблицу
pivot_purpose

debt,0,1,percent,trust
purpose_category,,,,
операции с автомобилем,3898,401,10.287327,0.905526
операции с недвижимостью,10024,780,7.781325,1.197153
получение образования,3638,369,10.142936,0.918416
проведение свадьбы,2156,183,8.487941,1.097491


Проанализируем. 
Наиболее надежная цель кредита, это `операции с недвижимостью`+20%. А самые ненадежные - `операции с автомобилем` и `получение образования` -9% и -8% соответственно. 
Наверняка это связано с особенностью выдачи кредита под разные цели. При выдаче кредита под операцию с недвижимостью больше залог, больше первоначальный взнос и люди подходят к возврату кредита более ответственно. 

 **Вывод**

Исследование надежности заемщиков имеет большое прикладное значение. Можно пролучить нетривиальные выводы. Самые надежные клиенты это вдова/вдовец, без детей, имеющие доход от 30001 до 50000 и берущие кредит для операций с недвижимостью. А самый ненадежный клиент это не женат / не замужем, с 4-мя детьми с доходом до 30000 и берущий кредит на покупку автомобиля. (Сложно себе представить таких клиентов).Все остальные варианты по надежности возврата кредита в срок располагаются между ними.